In [1]:
from gensim.models import Word2Vec
import multiprocessing
import pandas as pd
from tqdm import tqdm_notebook

In [17]:
def add_w2v_feat(data_frame, feat, L=5):
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    w2v_feat_list = []
    model = Word2Vec(data_frame[feat].values.tolist(), size=L, window=10, min_count=1,
                                     workers=multiprocessing.cpu_count(), iter=10)
    new_all = pd.DataFrame()
    for m, t in tqdm_notebook(enumerate(feat)):
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)

        w2c_list = [f'w2v_{t}_{n}' for n in range(L)]
        tmp_df.columns = [t] + w2c_list
        data_frame = data_frame.merge(tmp_df, on=t, how='left')
        w2v_feat_list += w2c_list
    return data_frame, w2v_feat_list

In [2]:
train_df = pd.read_parquet('./tmp_data/train_df.parquet')
test_df = pd.read_parquet('./tmp_data/test_df.parquet')

In [6]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]

In [28]:
df = pd.concat([train_df[features], test_df[features]])

In [30]:
w2v_feats = ['var_34', 'var_80', 'var_9', 'var_108', 'var_13', 'var_94',
       'var_184', 'var_1', 'var_53', 'var_22', 'var_2', 'var_12',
       'var_146', 'var_99', 'var_133', 'var_166', 'var_21', 'var_6',
       'var_165', 'var_154', 'var_174', 'var_198', 'var_76', 'var_78',
       'var_109', 'var_173', 'var_148', 'var_75', 'var_179', 'var_33',
       'var_92', 'var_123', 'var_190', 'var_127', 'var_169', 'var_87',
       'var_164', 'var_170', 'var_122', 'var_191']

In [31]:
for i in tqdm_notebook(w2v_feats):
    if df[i].dtype != 'object':
        df[i] = df[i].astype(str)

In [32]:
df, w2v_feat_list = add_w2v_feat(df, w2v_feats, L=5)

/data/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [33]:
df = df[w2v_feats + w2v_feat_list]

In [34]:
df.to_parquet('./tmp_data/w2v_feats.parquet')